In [3]:
!pip install mediapipe opencv-python pandas scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 1.2 MB/s 


In [4]:
# import the necessary packages

from zipfile import ZipFile
from matplotlib import pyplot as plt
from imutils import face_utils
import numpy as np
import argparse
import imutils
import mediapipe as mp
import os
import sys
import os,cv2,math,glob,random
import scipy.io as sio
from math import cos, sin
from pathlib import Path
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split


In [5]:
!wget  http://www.cbsr.ia.ac.cn/users/xiangyuzhu/projects/3DDFA/Database/AFLW2000-3D.zip



--2022-06-17 19:12:11--  http://www.cbsr.ia.ac.cn/users/xiangyuzhu/projects/3DDFA/Database/AFLW2000-3D.zip
Resolving www.cbsr.ia.ac.cn (www.cbsr.ia.ac.cn)... 159.226.21.60
Connecting to www.cbsr.ia.ac.cn (www.cbsr.ia.ac.cn)|159.226.21.60|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87413900 (83M) [application/x-zip-compressed]
Saving to: ‘AFLW2000-3D.zip’

AFLW2000-3D.zip     100%[===================>]  83.36M   285KB/s    in 5m 6s   

2022-06-17 19:17:18 (279 KB/s) - ‘AFLW2000-3D.zip’ saved [87413900/87413900]



In [6]:
# Loading AFLW2000 Data
%%capture
!unzip /content/AFLW2000-3D.zip

In [7]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_drawing_styles = mp.solutions.drawing_styles





In [8]:
img_list= []
img_mat = []
for images in glob.glob('/content/AFLW2000/*.jpg'):
   
    # check if the image ends with png
    img_list.append(images)

img_list.sort()
print(len(img_list))
print(img_list[0:3])

2000
['/content/AFLW2000/image00002.jpg', '/content/AFLW2000/image00004.jpg', '/content/AFLW2000/image00006.jpg']


In [9]:
img_mat = []
for images in glob.glob('/content/AFLW2000/*.mat'):
   
    # check if the image ends with png
    img_mat.append(images)

img_mat.sort() 
print(len(img_mat))
print(img_mat[0:3])

2000
['/content/AFLW2000/image00002.mat', '/content/AFLW2000/image00004.mat', '/content/AFLW2000/image00006.mat']


In [10]:
mp_face_mesh = mp.solutions.face_mesh
mp_face_detection = mp.solutions.face_detection
mp_holistic = mp.solutions.holistic 


In [12]:
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)


In [13]:
# Colab has a problem with holistic process

In [15]:
# array = np.array([np.linspace(1,936, num= 936)])
# ypr_array = np.array([np.linspace(1,3, num= 3)])
# for indx, img in enumerate(img_list):

#         all_imgs = []
#         all_ypr = []
#         image = cv2.imread(img)
#         image_height, image_width, _ = image.shape
# #         Convert the BGR image to RGB before processing.
#         results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

#         try:
           
#             face  = results.face_landmarks.landmark
#             NoseX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x  *  image_width
#             NoseY =results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y  * image_height
#             LfeoX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].x * image_width
#             LfeoY = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].y * image_height
            
#             dist = math.dist([NoseX, NoseY], [LfeoX, LfeoY])
        

            
#             face_row = list(np.array([[   ((landmark.x * image_width)-NoseX)/dist, 
#                                             ((landmark.y*image_height)-NoseY)/dist] for landmark in face]).flatten())
            
#             mat_file = sio.loadmat(img_mat[indx])
#             # extracting the labels 3 angels
#             pose_para = mat_file["Pose_Para"][0][:3]
#             pitch = pose_para[0]
#             yaw = pose_para[1]
#             roll = pose_para[2] 
#             all_ypr.append(pitch)
#             all_ypr.append(yaw)
#             all_ypr.append(roll)
#             all_ypr_array = np.array(all_ypr)
#             ypr_array = np.vstack([ypr_array,all_ypr_array.T])

#             # with open('landmarks_data.csv', mode='a', newline='') as f:
#             #     csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#             #     csv_writer.writerow(face_row) 
                
#         except:
#             pass
        

In [16]:
image = cv2.imread(img_list[0])

In [18]:
image.shape

(450, 450, 3)

In [25]:
array = np.array([np.linspace(1,936, num= 936)])
ypr_array = np.array([np.linspace(1,3, num= 3)])
for indx , img in enumerate(img_list) :
  
  all_imgs = []
  all_ypr = []
  image = cv2.imread(img)
  image = imutils.resize(image, width=450)
  # cv2_imshow(image)
  result = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  face = result.multi_face_landmarks

  try:
    for landm in face[0].landmark:
      x = landm.x
      y = landm.y
      # note: the x and y values are Normalized from 0 to 1 and we need to denormalize them
      relative_x = int(x * image.shape [1])
      relative_y = int(y * image.shape [0])
      all_imgs.append(np.array([relative_x]))
      all_imgs.append(np.array([relative_y]))
    all_imgs_arr = np.array(all_imgs)
    array = np.vstack([array,all_imgs_arr.T])
    

    mat_file = sio.loadmat(img_mat[indx])
    # extracting the labels 3 angels
    pose_para = mat_file["Pose_Para"][0][:3]
    pitch = pose_para[0]
    yaw = pose_para[1]
    roll = pose_para[2] 
    all_ypr.append(pitch)
    all_ypr.append(yaw)
    all_ypr.append(roll)
    all_ypr_array = np.array(all_ypr)
    ypr_array = np.vstack([ypr_array,all_ypr_array.T])


  except:
    pass


In [26]:
array.shape

(1854, 936)

In [27]:
ypr_array.shape

(1854, 3)

In [28]:
array= np.delete(array, (0) , axis=0)
df = pd.DataFrame(array)
df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,926,927,928,929,930,931,932,933,934,935
0,218.0,309.0,220.0,287.0,220.0,291.0,215.0,253.0,220.0,279.0,...,251.0,218.0,246.0,221.0,243.0,225.0,299.0,212.0,304.0,208.0
1,198.0,288.0,187.0,266.0,201.0,274.0,196.0,245.0,187.0,259.0,...,235.0,228.0,230.0,230.0,225.0,231.0,263.0,225.0,267.0,222.0


In [29]:
len(df)

1853

In [30]:
ypr_array= np.delete(ypr_array, (0) , axis=0)
# columns = ['x1','y1','x2','y2','x3','y3','x4','y4','x5','y5']
df_ypr = pd.DataFrame(ypr_array, columns=['pitch','yaw','roll'])
df_ypr.head(2)

,pitch,yaw,roll
0,-0.399231,0.018227,0.085676
1,0.470065,1.189533,0.300959


In [31]:
len(df_ypr)

1853

In [32]:
df_copy = df.copy()
df_copy['yaw'] = df_ypr['yaw']
df_copy['pitch'] = df_ypr['pitch']
df_copy['roll'] = df_ypr['roll']
df_copy.head(2)

,0,1,2,3,4,5,6,7,8,9,...,929,930,931,932,933,934,935,yaw,pitch,roll
0,218.0,309.0,220.0,287.0,220.0,291.0,215.0,253.0,220.0,279.0,...,221.0,243.0,225.0,299.0,212.0,304.0,208.0,0.018227,-0.399231,0.085676
1,198.0,288.0,187.0,266.0,201.0,274.0,196.0,245.0,187.0,259.0,...,230.0,225.0,231.0,263.0,225.0,267.0,222.0,1.189533,0.470065,0.300959


In [33]:
# features
x= np.array(df)
# pitch label
y_p=df_ypr['pitch']
# yaw label
y_y=df_ypr['yaw']
# roll label
y_r=df_ypr['roll']


In [200]:
# PITCH 

X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(x, y_p, test_size=0.2, random_state=42, shuffle= True)

X_val_p, X_test_p, y_val_p, y_test_p = train_test_split(X_test_p, y_test_p, test_size=0.5, random_state=42, shuffle= True)

In [141]:
# YAW 

X_train_y, X_test_y, y_train_y, y_test_y = train_test_split(x, y_y, test_size=0.2, shuffle=True,random_state=42)

X_val_y, X_test_y, y_val_y, y_test_y = train_test_split(X_test_y, y_test_y, test_size=0.5, shuffle=True,random_state=42)


In [207]:
# Roll 

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(x, y_r, test_size=0.2,shuffle=True, random_state=42)

X_val_r, X_test_r, y_val_r, y_test_r = train_test_split(X_test_r, y_test_r, test_size=0.5,shuffle=True, random_state=42)

In [143]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer, PowerTransformer, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge , ridge_regression,Lasso
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor, VotingRegressor,BaggingRegressor,ExtraTreesRegressor
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score 
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.datasets import make_regression
import sklearn.linear_model
from sklearn.impute import KNNImputer
from sklearn import metrics


# Yaw model 

In [144]:

rf_model = make_pipeline(StandardScaler(),PCA(n_components=100, whiten=True),RandomForestRegressor())
rf_model.fit(X_train_y, y_train_y)
rf_predict_test = rf_model.predict(X_test_y)

In [150]:
# Test error
score_test = round(rf_model.score(X_test_y, y_test_y), 4)
mae_rf = metrics.mean_absolute_error(y_test_y, rf_predict_test)
mse_rf = metrics.mean_squared_error(y_test_y, rf_predict_test)

In [224]:
print(f"Test score : {score_test}")
print(f"mean_absolute_error  : {mae_rf}")
print(f"mean_squared_error  : {mse_rf}")

Test score : 0.9494
mean_absolute_error  : 0.07750679889354539
mean_squared_error  : 0.012835268721898255


In [225]:
import pickle 

with open('yaw_model.pkl', 'wb') as f:
    pickle.dump(rf_model, f)



# Pitch Model

In [ ]:

rf_model_p = make_pipeline(RobustScaler(),PCA(n_components=0.99, whiten=True),RandomForestRegressor())
rf_model_p.fit(X_train_p, y_train_p)
rf_predict_test_p = rf_model_p.predict(X_test_p)

In [ ]:

# Test error
score_test_p = round(rf_model_p.score(X_test_p, y_test_p), 4)
mae_rf_p = metrics.mean_absolute_error(y_test_p, rf_predict_test_p)
mse_rf_p = metrics.mean_squared_error(y_test_p, rf_predict_test_p)

In [ ]:
print(f"Test score : {score_test_p}")
print(f"mean_absolute_error  : {mae_rf_p}")
print(f"mean_squared_error  : {mse_rf_p}")

Validation score : 0.4451
Test score : 0.802
mean_absolute_error  : 0.08216324800090855
mean_squared_error  : 0.012748234842533514


In [226]:
import pickle 

with open('pitch_model.pkl', 'wb') as f:
    pickle.dump(rf_model_p, f)


# Roll Model

In [220]:

rf_model_r = make_pipeline(RobustScaler(),PCA(n_components=100, whiten=True),RandomForestRegressor())
rf_model_r.fit(X_train_r, y_train_r)
rf_predict_test_r = rf_model_r.predict(X_test_r)

In [221]:

# Test error
score_test_r = round(rf_model_r.score(X_test_r, y_test_r), 4)
mae_rf_r = metrics.mean_absolute_error(y_test_r, rf_predict_test_r)
mse_rf_r = metrics.mean_squared_error(y_test_r, rf_predict_test_r)

In [222]:
print(f"Test score : {score_test_r}")
print(f"mean_absolute_error  : {mae_rf_r}")
print(f"mean_squared_error  : {mse_rf_r}")

Test score : 0.811
mean_absolute_error  : 0.07039624950712568
mean_squared_error  : 0.018485500632624396


In [227]:
import pickle 

with open('roll_model.pkl', 'wb') as f:
    pickle.dump(rf_model_r, f)


In [230]:
with open('yaw_model.pkl', 'rb') as f:
    yaw_model = pickle.load(f)

In [231]:
with open('pitch_model.pkl', 'rb') as f:
    pitch_model = pickle.load(f)

In [232]:
with open('roll_model.pkl', 'rb') as f:
    roll_model = pickle.load(f)

In [242]:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)

    # Draw the face mesh annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_face_landmarks:
      for face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_tesselation_style())
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_iris_connections_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Face Mesh', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()